In [ ]:
import pandas as pd
import nltk
import re, pprint
from nltk import word_tokenize

In [185]:
egt={1:"West Aurora School District 129, together with support of the United States Department of Labor and the American Health Information Management Association Foundation, have developed Illinois's first Federally Registered Apprenticeship Program in a health field at the secondary education level. The apprenticeship program will focus on assisting high school graduates with Certified Nursing Assistant credentials by advancing their skills and assisting students with the transition from being a student to becoming a highly skilled employee. Local employers will provide graduates with full-time employment and a mentor who will work with district staff to monitor skill development and provide support. West Aurora School District 129 will be supporting its students past graduation through a Registered Apprenticeship program. This program will focus on bringing employers, community members and education together for successful student transition and extended skill development - Cynthia Garcia, Director of Extended Learning and Career Pathways. The apprenticeship program will provide apprentices with transition support of various kinds through district staff and a mentor provided by the employer. District 129 will provide the required training for the apprenticeship program before students graduate from high school to make their transition year as a new employee manageable. Upon completion of the apprenticeship, the apprentice will receive tuition assistance to support lifelong learning and a Completion Certificate from the U.S. Department of Labor, Office of Apprenticeship. Partnering employers will have the opportunity to assist these apprentices by increasing their mastery of needed skills to work as a CNA. The benefit for the partnering employer is a highly skilled and competent staff that provides quality individualized care for their residence. Employers will also receive a certificate from the U.S. Department of Labor, Office of Apprenticeship showing they are a Federally Registered Apprenticeship Training site and an education partner."}

## Basic strategy:
- Obtain the named entities in the articles
- collocation of "organize" or its synonyms with "apprenticeship" or NE

In [ ]:
def inpreprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [ ]:
from nltk.chunk.regexp import tag_pattern2re_pattern

In [ ]:
tag_pattern2re_pattern('<DT>?<JJ.>?*<NN.>*<developed>*<JJ.*>?*<NN.*>')

In [ ]:
grammar = "NP: {<DT>?<JJ.>*<NN.*>+}" # This structure seems to work well
#grammar = "NP: {<DT>?<JJ.>?*<NN.>*<.*>*<JJ.*>*<NN.*>+} " #{<DT>?<NN.><JJ.>?*<.*><JJ.*>?*<NN.*>}
# This structure seems to work well

In [ ]:
cp = nltk.RegexpParser(grammar)

In [ ]:
def letter_parser(letter):
    '''
    input - letter is a dict 
    output - is a dict of dict with all NP from a letter
    
    '''
    dat = dict() 
    
    let_id = list(egt.items())[0][0]
    let_val = list(egt.items())[0][1]
    
    eg=inpreprocess(let_val)  # processing the letter
    dt = []
    for it1 in eg:
        
        result = cp.parse(it1)
        for items in range(len(result)):
            print(ne_chunk(result[items]))
            if hasattr(result[items],'label'):
                if(result[items]).label()=='NP':
                    dt.append(result[items])
    dat[let_id]=dt
    
    return dat

In [ ]:
letter_parser(egt)

In [ ]:
eg[1]

## Named entity recognition

In [217]:
import pandas as pd

In [219]:
from nltk.chunk import ne_chunk
from nltk.chunk.regexp import ChunkString

In [200]:
def sub_leaves(tree, label):
    return [t.leaves() for t in tree.subtrees(lambda s: s.label() == label)]

In [289]:
mygrammar = "NP: {<NNP>+}"

In [333]:
def nerTest(mydataframe, grammar):
    '''
    input - pandas dataframe 
    output - is a dict of dict with all NP from a letter
    
    '''
    dat = dict() 
    dat_per = dict()
    cp = nltk.RegexpParser(grammar)
    
    for ind in mydataframe.index:
        let_id = ind
        let_val = mydataframe.loc[ind,'Article Text']
    
        eg=inpreprocess(let_val)  # processing the letter
        dt = []
        dt1 = []
        for it1 in eg:
            result = cp.parse(it1)
            for items in range(len(result)):
            #print(result[items]) #
                if hasattr(result[items],'label'):
                    if(result[items]).label()=='NP':
                        tree1=ne_chunk(result[items]) #,binary=True)
                        sl = sub_leaves(tree1,'ORGANIZATION')
                        print(sl)
                        break
                        sl2 = sub_leaves(tree1,'PERSON')
                        if sl != []:
                            dt.append(sl)
                        if sl2 != []:
                            dt1.append(sl2)
        dat[let_id]=dt
        dat_per[let_id]=dt1
    
    return (dat,dat_per)

## Implementing on "Articles.csv" data

In [248]:
dat=pd.read_csv("articles.csv", sep =',', encoding = 'latin-1')

In [323]:
dat2=dat.head(5)

In [335]:
a=[[('Palmgren', 'NNP')]]

In [294]:
org1, per = nerTest(dat,mygrammar)

In [311]:
org_dat=pd.DataFrame.from_dict(org1, orient='index')

In [312]:
org_dat.to_csv('org_names.csv')

In [306]:
a= org1[0][0]